### Experimental paradigm

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from neuropy import plotting
import seaborn as sns
import mab_subjects
from statplotannot.plots import fix_legend

exps = mab_subjects.mostly_unstruc.allsess[1:2] + mab_subjects.mostly_struc.allsess[3:4]
mov_mean = lambda x, w: np.convolve(x, np.ones(w) / w, mode="valid")

# Indexes for good examples
# Sterling: 21022, 22522
# Debruyne: 54821, 56321
# 30900 32400
# 47054 48554

window_size = 30
Titles = ["Unstructured", "Structured"]

fig = plotting.Fig(7, 4, fontsize=10)

for i, exp in enumerate(exps):

    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100)
    probs = task.probs
    n_trials = probs.shape[0]

    # n1 = np.random.randint(int(0.8 * n_trials), probs.shape[0])
    n1 = 30900 if i == 0 else 47054
    n2 = n1 + 1500
    print(n1, n2)

    prob1 = probs[:, 0][n1:n2]
    prob2 = probs[:, 1][n1:n2]

    subfig = fig.add_subfigure(fig.gs[i])
    axs = subfig.subplots(2, 1, height_ratios=[3, 1], sharex=True)

    ax = axs[0]
    ax.plot(prob1, label="Left", color="blue")
    ax.plot(prob2, label="Right", color="orange")
    # ax.plot(prob1 - prob2, label="P(L) - P(R)", color="purple")
    # ax.spines["right"].set_visible(True)
    ax.set_yticks([0.1, 0.50, 1.00])
    ax.set_ylabel("P (reward)")
    ax.legend(loc="upper right")
    fix_legend(ax, only_labels=True)

    # ax2 = ax.twinx()
    ax2 = axs[1]

    rewards = task.rewards[n1:n2]
    choices = task.choices[n1:n2]
    choices_left = np.where(choices == 1, 1, 0)
    choices_right = np.where(choices == 2, 1, 0)

    reward_rate = mov_mean(rewards, window_size)
    choice_left_prob = mov_mean(choices_left, window_size)
    choice_right_prob = mov_mean(choices_right, window_size)

    # ax2.plot(reward_rate, label="Reward rate (20-trial MA)", color="red")
    ax2.plot(choice_left_prob, label="P(L)", color="#7a7a7a")
    # ax2.plot(choice_right_prob, label="P(R)", color="purple")
    # ax2.plot(rewards, label="Reward", color="green", alpha=0.5)
    ax2.set_ylabel("Choice\nP(Left)")
    ax2.set_xlabel("Trials")

    subfig.suptitle(Titles[i])

figpath = mab_subjects.figpath / "npdf"
fig.savefig(figpath / "expample_sessions")

### Performance across combinations

In [ ]:
from statplotannot.plots import SeabornPlotter, fix_legend
from mab_colors import Palette2Arm
import seaborn as sns
from statplotannot.plots import Fig
import mab_subjects

perf_df = mab_subjects.GroupData().perf_all_corr_uncorr.latest

fig = Fig(8, 5, fontsize=10)

for i, y in enumerate(["All", "Correlated", "Uncorrelated"]):
    ax = fig.subplot(fig.gs[1, i + 1])
    sns.lineplot(
        data=perf_df,
        x="trial_id",
        y=y,
        hue="grp",
        palette=Palette2Arm().as_dict(),
        errorbar="se",
    )
    fix_legend(ax)
    ax.set_ylim(0.4, 0.85)
    ax.set_ylabel("P(High)")
    ax.set_title(y)
    ax.grid(axis="y", zorder=-1, alpha=0.5)


figpath = mab_subjects.figpath / "npdf"
fig.savefig(figpath / "choice_performance_corr_uncorr.png", dpi=300)

### State inference parameters plot

In [ ]:
import seaborn as sns
from neuropy import plotting
import mab_subjects
from mab_colors import Palette2Arm

fig = plotting.Fig(8, 5, size=(8.5, 11), fontsize=10)

df = mab_subjects.GroupData().fit_si_sim.latest

hue_order = ["unstruc", "struc"]
plot_kw = dict(
    data=df,
    x="trial_id",
    hue="grp",
    hue_order=hue_order,
    palette=Palette2Arm().as_dict(),
    linewidth=1,
    errorbar="se",
    err_kws=dict(edgecolor="none"),
)

for i, block in enumerate(["all"]):
    ax = fig.subplot(fig.gs[i])
    sns.lineplot(y=f"perf_{block}", ax=ax, linestyle="solid", **plot_kw)
    sns.lineplot(y=f"sim_perf_{block}", ax=ax, linestyle="dashed", **plot_kw)

    ax.set_title(f"{block}: actual vs simulated")
    ax.set_ylabel("P (Optimal choice)")
    ax.set_ylim(0.45, 0.9)
    ax.set_xticks([1, 25, 50, 75, 100])
    ax.get_legend().remove()
    # ax.legend(['a','b','c','d'])

    # ax.legend(["unstruc - actual", "struc - actual", "unstruc - sim", "struc - sim"])

figpath = mab_subjects.figpath / "npdf"
fig.savefig(figpath / "si_actual_vs_simulated")

### Q learning fit

In [ ]:
import seaborn as sns
from neuropy import plotting
import mab_subjects
from mab_colors import Palette2Arm
import numpy as np

fig = plotting.Fig(8, 5, fontsize=10)

df = mab_subjects.GroupData().fit_qlearnH_sim.latest

hue_order = ["unstruc", "struc"]
plot_kw = dict(
    data=df,
    x="trial_id",
    hue="grp",
    hue_order=hue_order,
    palette=Palette2Arm().as_dict(),
    linewidth=1,
    errorbar="se",
    err_kws=dict(edgecolor="none"),
)

for i, block in enumerate(["all"]):
    ax = fig.subplot(fig.gs[i])
    sns.lineplot(y=f"perf_{block}", ax=ax, linestyle="solid", **plot_kw)
    sns.lineplot(y=f"sim_perf_{block}", ax=ax, linestyle="dashed", **plot_kw)

    ax.set_title(f"{block}: actual vs simulated")
    ax.set_ylabel("P (Optimal choice)")
    ax.set_ylim(0.45, 0.9)
    ax.set_xticks([1, 25, 50, 75, 100])
    # ax.get_legend().remove()
    # ax.legend(['a','b','c','d'])

    ax.legend(["unstruc - actual", "struc - actual", "unstruc - sim", "struc - sim"])
    ax.set_ylim(0.45, 0.9)
    ax.set_yticks([0.6, 0.8])

ax.legend_.remove()

figpath = mab_subjects.figpath / "npdf"
fig.savefig(figpath / "qlearnH_actual_vs_simulated")